# PyBADS Example 2: Non-box constraints

In this example, we will show how to set more complex constraints in PyBADS, besides a simple bounded box.

This notebook is Part 2 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads.bads import BADS

## 0. Constrained optimization

PyBADS naturally supports box constraints `lb` and `ub`, as we saw in the [previous example](./pybads_example_1_basic_usage.ipynb). However, some optimization problems might have more complex constraints over the variables. Formally, we may wish to solve the problem

$$
\mathbf{x}^\star = \arg\min_{\mathbf{x} \in \mathcal{X}} f(\mathbf{x})
$$
where $\mathcal{X} \subseteq \mathbb{R}^D$ is the admissible region for the optimization.

We can do this in PyBADS by providing a function `nonboxcons` that defines constraints *violation*, that is a function $g(\mathbf{x})$ which returns `True` if $\mathbf{x} \notin \mathcal{X}$, as demonstrated below.

## 1. Problem definition

We optimize [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in 2D as in the [previous example](./pybads_example_1_basic_usage.ipynb), but here we force the input to stay within a circle with unit radius.

Since we know the optimization region, we set tight box bounds `lb` and `ub` around the circle to further help the search.

Note that `nonboxcons` takes as input a $M$-by-$D$ array $\mathbf{x}_1, \ldots, \mathbf{x}_M$, where each $\mathbf{x} \in \mathbb{R}^D$, and outputs a $M$-by-1 `bool` array, where $M$ is an arbitrary number for the inputs. The $m$-th value of the output array is `True` if $\mathbf{x}_m$ *violates* the constraint, `False` otherwise.

In [2]:
def rosenbrocks_fcn(x):
    """Rosenbrock's 'banana' function in any dimension."""
    x_2d = np.atleast_2d(x)
    return np.sum(100 * (x_2d[:, 0:-1]**2 - x_2d[:, 1:])**2 + (x_2d[:, 0:-1]-1)**2, axis=1)

x0 = np.array([[0, 0]]);      # Starting point
lb = np.array([[-1, -1]])     # Lower bounds
ub = np.array([[1, 1]])       # Upper bounds

def circle_constr(x):
    """Return constraints violation outside the unit circle."""
    x_2d = np.atleast_2d(x)
    # Note that nonboxcons assumes the function takes a 2D input 
    return np.sum(x_2d**2, axis=1) > 1

## 2. Run the optimization

We initialize `bads` with the non-box constraints defined by `nonboxcons`. Note that we also still specify standard box constraints `lb` and `ub`, as this will help the search.

Incidentally, here BADS will complain because the plausible bounds are not specified explicitly. In the absence of plausible bounds, BADS will create them based on the lower/upper bounds instead. Generally, you should specify the plausible bounds.

In [3]:
bads = BADS(rosenbrocks_fcn, x0, lb, ub, nonbondcons=circle_constr)
x_min, fval = bads.optimize()

bads:TooCloseBounds: For each variable, hard and plausible bounds should not be too close. Moving plausible bounds.
Variables (index) defined with periodic boundaries: []
Beginning optimization of a DETERMINISTIC objective function

 Iteration f-count     f(x)     MeshScale     Method     Actions
     0         3       1.000000      1.000000            Uncertainty test
     0         7       1.000000      1.000000     Initial mesh       Initial points


c:\users\luigi\documents\github\gpyreg\gpyreg\covariance_functions.py:376: RuntimeWarning: invalid value encountered in add
  plausible_lower_bounds[i_nan] + plausible_upper_bounds[i_nan]


     0        11       1.000000      0.500000     Refine grid       Train
     1        14       0.561451      0.500000     Successful search (('ES-ell', 1))       
     1        19       0.299069      0.500000     Incremental search (('ES-wcm', 1))       
     1        23       0.299069      0.250000     Refine grid       Train
     2        25       0.085135      0.250000     Successful search (('ES-wcm', 1))       
     2        26       0.000000      0.250000     Incremental search (('ES-wcm', 1))       
     2        33       0.000000      0.125000     Refine grid       
     3        39       0.000000      0.062500     Refine grid       


c:\users\luigi\documents\github\pybads\pybads\bads\bads.py:1760: RuntimeWarning: divide by zero encountered in true_divide
  zscore = zscore / gp_ys


bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
     4        45       0.000000      0.031250     Refine grid       
     5        48       0.000000      0.031250     Incremental search (('ES-wcm', 1))       
bads: The optimization is stalling, decreasing further the mesh size
     5        51       0.000000      0.007812     Refine grid       


c:\users\luigi\documents\github\pybads\pybads\bads\bads.py:1555: RuntimeWarning: divide by zero encountered in true_divide
  gamma_z = (self.optim_state['f_target'] - self.sufficient_improvement - f_mu) / fs


bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads: The optimization is stalling, decreasing further the mesh size
     6        57       0.000000      0.001953     Refine grid       
Optimization terminated: change in the function value less than options.TolFun.
Function value at minimum: 1.3359650640840094e-07



## 3. Results and conclusions

In [4]:
print(f"BADS minimum at: x = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {bads.function_logger.func_count-1}, time: {round(bads.optim_state['total_time'], 2)} s")

BADS minimum at: x = [0.99998158 0.99999966], fval = 1.336e-07
total f-count: 57, time: 3.58 s


The true global minimum of the Rosenbrock function under these constraints is at $\textbf{x}^\star = [0.786,0.618]$, where $f^\star = 0.046$.

### Remarks

- While in theory `nonboxcons` can receive any arbitrary constraints, in practice PyBADS will likely work well only within relatively simple domains (e.g., simple convex regions), as the current version of (Py)BADS uses a simple heuristic to reject samples outside the admissible region.
- In particular, PyBADS does *not* support equality constraints (e.g., of the form $x_1 + x_2 + x_3 = 1$).

## Example 2: Full code

See [here](./src/pybads_example_2_nonbox_constraints.py) for a Python file with the code used in this example, with no extra fluff.